In [2]:
import requests
import requests_cache
from matplotlib import pyplot as plt
import lxml.html
import pandas as pd
import re
import json 
import lxml.html as lx
import urllib
from urllib2 import Request, urlopen, HTTPError
from urlparse import urlunparse, urlparse, urljoin
import numpy

In [3]:
def fetch_page(url):
    response = requests.get(url)
    response.raise_for_status()
    return response

url="https://www.zillow.com/browse/homes/ca/"
requests_cache.install_cache("cache")
page = fetch_page(url)
html = lx.fromstring(page.text)
county_html = html.xpath('//main/div/div/ul/li/a')
county = [i.text_content() for i in county_html]
county_name = pd.DataFrame({"RegionName":[" ".join(k) for k in [i.split(" ")[:-2] for i in county]]})

In [4]:
#Import Housing Data
import os.path


def dataframe(i = 0):
    
    """Read datasets
    Please change the file path to fit to your computer
    Input:the name of dataset //dataname   
    """
    
    file_path = "/Users/HUI/Dropbox/16_UCDavis/17W_STA141B/05_Project/Housing_Data"
    file_name = ["County_MedianSoldPrice_AllHomes.csv","County_MedianListingPrice_AllHomes.csv","County_MedianListingPrice_1Bedroom.csv",
                 "County_MedianListingPrice_2Bedroom.csv","County_MedianListingPrice_3Bedroom.csv","County_MedianListingPrice_4Bedroom.csv",
                 "County_MedianListingPrice_5BedroomOrMore.csv","County_MedianListingPrice_CondoCoop.csv","County_MedianListingPrice_DuplexTriplex.csv",
                 "County_MedianListingPrice_Sfr.csv"]
    dataframe = pd.read_csv(os.path.join(file_path,file_name[i]),header=0)
    CA_dataframe = dataframe[dataframe["State"]== "CA"]
    CA_dataframe = CA_dataframe.reset_index(drop=True)
    return(CA_dataframe)

###name of eachdata
dataname=["sold","list","one","two","thre","four","five","cond","trip","sfr"]

###save all of dataset to CA dict.
CA = {i: dataframe(dataname.index(i)) for i in dataname}

In [5]:
#All Home Sold time-series data 1996-2016
CA["sold"] = pd.merge(pd.DataFrame(county_name),CA["sold"], how='left', on=['RegionName'])
CA_sold_t = CA["sold"].transpose()
CA_sold_t = CA_sold_t.rename(columns=lambda i: CA["sold"]["RegionName"][i])[7:]
date = pd.date_range('1996-04', '2016-07', freq='M')
CA_sold_t['date'] = date

#sold price time-series data
CA_sold_t = CA_sold_t.set_index('date')
CA_sold_t.head()


,Los Angeles,Orange,San Diego,Riverside,San Bernardino,Santa Clara,Alameda,Sacramento,Contra Costa,Fresno,...,Glenn,Plumas,Colusa,Mariposa,Inyo,Trinity,Mono,Modoc,Sierra,Alpine
date,,,,,,,,,,,,,,,,,,,,,
1996-04-30,162225,191525,158225,114550,100250,234325,196275,120425,192750,94850,...,67000,NaN,NaN,NaN,121471,53000,77500,NaN,NaN,NaN
1996-05-31,162925,197550,158125,112500,101700,238575,196900,121675,203000,95475,...,65823.5,NaN,97230.8,115000,NaN,NaN,78970.6,NaN,NaN,NaN
1996-06-30,164500,197875,158925,115000,103450,238875,202225,122175,209050,95475,...,NaN,NaN,NaN,NaN,119077,51950,NaN,NaN,NaN,NaN
1996-07-31,164550,198050,158700,113975,101850,241875,203700,122175,207150,96550,...,NaN,NaN,NaN,94230.8,112882,NaN,81538.5,NaN,NaN,NaN
1996-08-31,163300,199300,157300,113225,101650,241200,202250,121075,202500,96625,...,NaN,80000,NaN,NaN,NaN,82300,98470.6,NaN,NaN,NaN


In [77]:
#Different type of house average yr 2015/2016
def aver(i="one"):
    data = CA[i]
    CA_2015 = data.iloc[:,[k.__contains__("2015") for k in data.keys()]]
    CA_2016 = data.iloc[:,[k.__contains__("2016") for k in data.keys()]]
    CA_mean = pd.DataFrame({i+"_15":CA_2015.mean(1),i+"_16":CA_2016.mean(1)})
    CA_ave_name = pd.DataFrame({"RegionName":data["RegionName"]})
    CA_ave = pd.concat([CA_ave_name,CA_mean],axis=1)
    return CA_ave
CA_ave=[aver(i) for i in dataname]


#average price in yr 25/26 for different type of house/and all homes
diff_ave = reduce(lambda left,right: pd.merge(left,right,how="left",on='RegionName'), CA_ave)
diff_ave.head()

,RegionName,sold_15,sold_16,list_15,list_16,one_15,one_16,two_15,two_16,thre_15,...,four_15,four_16,five_15,five_16,cond_15,cond_16,trip_15,trip_16,sfr_15,sfr_16
0,Los Angeles,498737.500000,518508.333333,537666.666667,587328.666667,363468.333333,409424.500000,433116.583333,469545.750000,497107.916667,...,659808.333333,701394.000000,1.038750e+06,1.176233e+06,452570.833333,488478.750000,664705.500000,7.312750e+05,576524.500000,629662.333333
1,Orange,601020.833333,623150.000000,675932.083333,705890.000000,279250.000000,303945.833333,417687.500000,441220.833333,639658.333333,...,841700.000000,883811.583333,1.382648e+06,1.439552e+06,422483.333333,446204.166667,924108.583333,1.120979e+06,787609.750000,820343.541667
2,San Diego,472187.500000,489250.000000,538650.000000,575525.000000,276475.000000,319328.625000,360778.166667,396704.083333,494828.166667,...,684390.666667,716899.958333,1.093432e+06,1.184135e+06,376438.583333,410828.333333,678408.333333,7.657750e+05,609515.250000,641470.583333
3,Riverside,309068.750000,320437.500000,344257.500000,361732.458333,145399.875000,145940.750000,227945.750000,237962.041667,327233.333333,...,394170.750000,413642.833333,4.613150e+05,4.925966e+05,273656.666667,280891.666667,352866.583333,3.872125e+05,356255.250000,376051.708333
4,San Bernardino,264327.083333,272666.666667,271312.500000,291733.333333,NaN,NaN,172779.083333,185468.166667,255624.833333,...,367520.833333,385991.166667,4.659978e+05,4.931395e+05,259899.916667,271735.416667,295638.125000,3.068167e+05,272363.916667,293408.333333


In [106]:
file_path = "/Users/HUI/Dropbox/16_UCDavis/17W_STA141B/05_Project/"
def readdata(file_name):
             #,mergedata=diff_ave):

    dataframe = pd.read_csv(os.path.join(file_path,file_name),header=0)
    
    return(dataframe)

Unemp_frame = readdata("Unemploy_Rate.csv")
Unemp_frame["County"] = [" ".join(i.split(" ")[:-1]) for i in Unemp_frame["County"]]
Unemp_frame.head()




,County,Unemploy_Rate
0,Alameda,4.3
1,Alpine,5.9
2,Amador,6.3
3,Butte,6.9
4,Calaveras,6.2


In [182]:
Pop_frame = readdata("population.csv")
Pop_frame = Pop_frame[["County","Population"]]
Pop_frame.head()

,County,Population
0,Alameda,1638215
1,Alpine,1110
2,Amador,37001
3,Butte,225411
4,Calaveras,44828


In [183]:
Unem_diff=pd.merge(diff_ave,Unemp_frame,how="left",left_on = "RegionName",right_on = "County")
Unem_diff

,RegionName,sold_15,sold_16,list_15,list_16,one_15,one_16,two_15,two_16,thre_15,...,five_15,five_16,cond_15,cond_16,trip_15,trip_16,sfr_15,sfr_16,County,Unemploy_Rate
0,Los Angeles,4.987375e+05,5.185083e+05,5.376667e+05,5.873287e+05,363468.333333,409424.500000,4.331166e+05,4.695458e+05,4.971079e+05,...,1.038750e+06,1.176233e+06,4.525708e+05,4.884788e+05,6.647055e+05,7.312750e+05,5.765245e+05,6.296623e+05,Los Angeles,5.7
1,Orange,6.010208e+05,6.231500e+05,6.759321e+05,7.058900e+05,279250.000000,303945.833333,4.176875e+05,4.412208e+05,6.396583e+05,...,1.382648e+06,1.439552e+06,4.224833e+05,4.462042e+05,9.241086e+05,1.120979e+06,7.876098e+05,8.203435e+05,Orange,4.1
2,San Diego,4.721875e+05,4.892500e+05,5.386500e+05,5.755250e+05,276475.000000,319328.625000,3.607782e+05,3.967041e+05,4.948282e+05,...,1.093432e+06,1.184135e+06,3.764386e+05,4.108283e+05,6.784083e+05,7.657750e+05,6.095152e+05,6.414706e+05,San Diego,4.8
3,Riverside,3.090688e+05,3.204375e+05,3.442575e+05,3.617325e+05,145399.875000,145940.750000,2.279458e+05,2.379620e+05,3.272333e+05,...,4.613150e+05,4.925966e+05,2.736567e+05,2.808917e+05,3.528666e+05,3.872125e+05,3.562552e+05,3.760517e+05,Riverside,6.0
4,San Bernardino,2.643271e+05,2.726667e+05,2.713125e+05,2.917333e+05,NaN,NaN,1.727791e+05,1.854682e+05,2.556248e+05,...,4.659978e+05,4.931395e+05,2.598999e+05,2.717354e+05,2.956381e+05,3.068167e+05,2.723639e+05,2.934083e+05,San Bernardino,5.7
5,Santa Clara,7.992500e+05,8.563750e+05,8.225505e+05,8.882302e+05,441718.666667,467069.833333,5.505682e+05,6.242908e+05,7.652370e+05,...,1.442610e+06,1.634610e+06,5.652396e+05,6.345125e+05,8.744629e+05,9.809928e+05,9.023479e+05,1.009151e+06,Santa Clara,3.8
6,Alameda,6.147958e+05,6.618083e+05,6.279979e+05,6.877866e+05,350766.666667,398109.916667,4.469397e+05,5.095338e+05,6.270787e+05,...,1.004167e+06,1.128108e+06,4.470845e+05,5.161860e+05,6.300936e+05,7.422946e+05,6.898739e+05,7.657369e+05,Alameda,4.3
7,Sacramento,2.751083e+05,2.914417e+05,3.066311e+05,3.241530e+05,NaN,163918.250000,NaN,2.231291e+05,2.737263e+05,...,4.531112e+05,4.828124e+05,NaN,2.089354e+05,3.221755e+05,3.381313e+05,3.170995e+05,3.328917e+05,Sacramento,5.5
8,Contra Costa,4.738750e+05,5.033750e+05,5.213989e+05,5.617979e+05,247643.750000,282395.375000,3.626432e+05,4.009770e+05,4.661188e+05,...,8.284602e+05,8.415768e+05,3.400662e+05,3.771677e+05,5.052538e+05,5.399998e+05,5.507790e+05,5.933021e+05,Contra Costa,4.5
9,Fresno,2.213333e+05,2.234167e+05,2.538533e+05,2.573312e+05,NaN,NaN,1.326749e+05,1.414245e+05,2.247429e+05,...,3.988833e+05,3.908638e+05,NaN,NaN,2.303375e+05,2.762271e+05,2.585845e+05,2.611608e+05,Fresno,10.2


In [184]:
Unem_diff_pop=pd.merge(Unem_diff,Pop_frame,how="left",left_on = "RegionName",right_on = "County")
Unem_diff_pop.head()

,RegionName,sold_15,sold_16,list_15,list_16,one_15,one_16,two_15,two_16,thre_15,...,cond_15,cond_16,trip_15,trip_16,sfr_15,sfr_16,County_x,Unemploy_Rate,County_y,Population
0,Los Angeles,498737.500000,518508.333333,537666.666667,587328.666667,363468.333333,409424.500000,433116.583333,469545.750000,497107.916667,...,452570.833333,488478.750000,664705.500000,7.312750e+05,576524.500000,629662.333333,Los Angeles,5.7,Los Angeles,10170292
1,Orange,601020.833333,623150.000000,675932.083333,705890.000000,279250.000000,303945.833333,417687.500000,441220.833333,639658.333333,...,422483.333333,446204.166667,924108.583333,1.120979e+06,787609.750000,820343.541667,Orange,4.1,Orange,3169776
2,San Diego,472187.500000,489250.000000,538650.000000,575525.000000,276475.000000,319328.625000,360778.166667,396704.083333,494828.166667,...,376438.583333,410828.333333,678408.333333,7.657750e+05,609515.250000,641470.583333,San Diego,4.8,San Diego,3299521
3,Riverside,309068.750000,320437.500000,344257.500000,361732.458333,145399.875000,145940.750000,227945.750000,237962.041667,327233.333333,...,273656.666667,280891.666667,352866.583333,3.872125e+05,356255.250000,376051.708333,Riverside,6.0,Riverside,2361026
4,San Bernardino,264327.083333,272666.666667,271312.500000,291733.333333,NaN,NaN,172779.083333,185468.166667,255624.833333,...,259899.916667,271735.416667,295638.125000,3.068167e+05,272363.916667,293408.333333,San Bernardino,5.7,San Bernardino,2128133


In [189]:
crim = readdata("crime.csv")
Unme_diff_pop_crim = pd.merge(Unem_diff_pop,crim.iloc[:,1:],how="left", on = "RegionName")
Unme_diff_pop_crim.head()

,RegionName,sold_15,sold_16,list_15,list_16,one_15,one_16,two_15,two_16,thre_15,...,cond_16,trip_15,trip_16,sfr_15,sfr_16,County_x,Unemploy_Rate,County_y,Population,CrimeCases
0,Los Angeles,498737.500000,518508.333333,537666.666667,587328.666667,363468.333333,409424.500000,433116.583333,469545.750000,497107.916667,...,488478.750000,664705.500000,7.312750e+05,576524.500000,629662.333333,Los Angeles,5.7,Los Angeles,10170292,43163.0
1,Orange,601020.833333,623150.000000,675932.083333,705890.000000,279250.000000,303945.833333,417687.500000,441220.833333,639658.333333,...,446204.166667,924108.583333,1.120979e+06,787609.750000,820343.541667,Orange,4.1,Orange,3169776,2967.0
2,San Diego,472187.500000,489250.000000,538650.000000,575525.000000,276475.000000,319328.625000,360778.166667,396704.083333,494828.166667,...,410828.333333,678408.333333,7.657750e+05,609515.250000,641470.583333,San Diego,4.8,San Diego,3299521,11931.0
3,Riverside,309068.750000,320437.500000,344257.500000,361732.458333,145399.875000,145940.750000,227945.750000,237962.041667,327233.333333,...,280891.666667,352866.583333,3.872125e+05,356255.250000,376051.708333,Riverside,6.0,Riverside,2361026,17148.0
4,San Bernardino,264327.083333,272666.666667,271312.500000,291733.333333,NaN,NaN,172779.083333,185468.166667,255624.833333,...,271735.416667,295638.125000,3.068167e+05,272363.916667,293408.333333,San Bernardino,5.7,San Bernardino,2128133,12956.0


In [190]:
Unme_diff_pop_crim.keys()

Index([u'RegionName', u'sold_15', u'sold_16', u'list_15', u'list_16',
       u'one_15', u'one_16', u'two_15', u'two_16', u'thre_15', u'thre_16',
       u'four_15', u'four_16', u'five_15', u'five_16', u'cond_15', u'cond_16',
       u'trip_15', u'trip_16', u'sfr_15', u'sfr_16', u'County_x',
       u'Unemploy_Rate', u'County_y', u'Population', u'CrimeCases'],
      dtype='object')

In [191]:
Unme_diff_pop_crim.drop(["County_x",'County_y'],axis=1, inplace=True)

In [192]:
Unme_diff_pop_crim.keys()

Index([u'RegionName', u'sold_15', u'sold_16', u'list_15', u'list_16',
       u'one_15', u'one_16', u'two_15', u'two_16', u'thre_15', u'thre_16',
       u'four_15', u'four_16', u'five_15', u'five_16', u'cond_15', u'cond_16',
       u'trip_15', u'trip_16', u'sfr_15', u'sfr_16', u'Unemploy_Rate',
       u'Population', u'CrimeCases'],
      dtype='object')

In [194]:
rank = readdata("county_all_rank.csv")
all_rank = pd.merge(Unme_diff_pop_crim,rank,how="left", left_on = "RegionName",right_on = "County")
all_rank.drop(['County'],axis=1, inplace=True)
all_rank.keys()

Index([u'RegionName', u'sold_15', u'sold_16', u'list_15', u'list_16',
       u'one_15', u'one_16', u'two_15', u'two_16', u'thre_15', u'thre_16',
       u'four_15', u'four_16', u'five_15', u'five_16', u'cond_15', u'cond_16',
       u'trip_15', u'trip_16', u'sfr_15', u'sfr_16', u'Unemploy_Rate',
       u'Population', u'CrimeCases', u'Score2016', u'Score2015', u'Rank2016',
       u'Highest Rank City 2016', u'Zip2016', u'Rank2015',
       u'Highest Rank City 2015', u'Zip2015', u'Number Students',
       u'Number of Ranked Elementary Schools',
       u'Number of Ranked Middle Schools', u'Number of Ranked High Schools'],
      dtype='object')

In [206]:
income = readdata("income.csv")
#income = income.drop(income.iloc[:,:3])
all_income = pd.merge(all_rank,income.iloc[:,3:],how="left", left_on = "RegionName",right_on = "county")
all_income.drop(['county'],axis=1, inplace=True)


Index([u'RegionName', u'sold_15', u'sold_16', u'list_15', u'list_16',
       u'one_15', u'one_16', u'two_15', u'two_16', u'thre_15', u'thre_16',
       u'four_15', u'four_16', u'five_15', u'five_16', u'cond_15', u'cond_16',
       u'trip_15', u'trip_16', u'sfr_15', u'sfr_16', u'Unemploy_Rate',
       u'Population', u'CrimeCases', u'Score2016', u'Score2015', u'Rank2016',
       u'Highest Rank City 2016', u'Zip2016', u'Rank2015',
       u'Highest Rank City 2015', u'Zip2015', u'Number Students',
       u'Number of Ranked Elementary Schools',
       u'Number of Ranked Middle Schools', u'Number of Ranked High Schools',
       u'total_income'],
      dtype='object')

In [136]:
CA_sold_t.to_csv("/Users/HUI/Dropbox/16_UCDavis/17W_STA141B/05_Project/CA_sold_t.csv")
diff_ave.to_csv("/Users/HUI/Dropbox/16_UCDavis/17W_STA141B/05_Project/diff_ave.csv")

In [207]:
all_income.to_csv("/Users/HUI/Dropbox/16_UCDavis/17W_STA141B/05_Project/all_data.csv")